# Testing out Fancy Gym

In [1]:
import fancy_gym
import gymnasium as gym
import numpy as np
import time

/home/kn/Code/Misc/mambaforge/envs/RLHF/lib/python3.8/site-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/home/kn/Code/Misc/mambaforge/envs/RLHF/lib/python3.8/site-packages/pkg_resources/__init__.py:2871: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)


pygame 2.5.2 (SDL 2.28.2, Python 3.8.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Tests for JSONification

In [12]:
from src.DataHandling.video_extractor import VideoExtractor
from src.DataHandling.simulator import Simulator
from src.DataHandling.database_manager import DBManager
from src.DataHandling.trajectory_pair import TrajectoryPair

env = gym.make('fancy/AirHockey-3dof-hit-v0', render_mode="rgb_array", width=600, height=400)

simulator = Simulator(env)
video_extractor = VideoExtractor(env)

frames, trajectory = simulator.simulate_for_n_seconds(seconds=3, seed=420)

initial_conditions, transitions = trajectory

db_manager = DBManager(debug=True)

print(transitions[100].action.dtype)

entry = TrajectoryPair(trajectory, trajectory).to_bson()

print(entry['trajectory1']['transitions'][0])
print(db_manager.add_entry(TrajectoryPair(trajectory, trajectory)))

t, error = db_manager.get_next_entry()

print(t.trajectory1.transitions[0])

db_manager.close_db()

#TODO: Fix new error "cannot encode object: 61, of type: <class 'numpy.int64'>"


float32
{'state': [0.967822957294397, -0.036734707638827635, 0.0, 0.0, 0.0, 0.0, -1.15570723, 1.30024401, 1.44280414, 0.0, 0.0, 0.0], 'state_dtype': 'float64', 'action': [[2.0005362033843994, 1.0828193426132202, -1.8704737424850464], [-1.1786351203918457, 1.1533701419830322, 0.3787243962287903]], 'action_dtype': 'float32', 'reward': 0, 'terminated': False, 'truncated': False, 'next_state': [0.967822957294397, -0.036734707638827635, 0.0, 0.0, 0.0, 0.0, -1.1586362388808724, 1.318277771631903, 1.3999360134545766, 0.1413515997735139, -0.13054657195903221, 0.1684058898543555], 'next_state_dtype': 'float64'}
(None, "cannot encode object: 61, of type: <class 'numpy.int64'>")


AttributeError: 'NoneType' object has no attribute 'trajectory1'

# Test for Recreation

In [2]:
from src.DataHandling.video_extractor import VideoExtractor
from src.DataHandling.simulator import Simulator

env = gym.make('fancy/AirHockey-3dof-hit-v0', render_mode="rgb_array", width=600, height=400)

simulator = Simulator(env)
video_extractor = VideoExtractor(env)

frames, trajectory = simulator.simulate_for_n_seconds(seconds=15, seed=69)

video_extractor.generate_video(frames=frames)

print("Second Video")
video_extractor.generate_video(frames=video_extractor.recreate_frames_from_trajectory(trajectory=trajectory))

frames2 = video_extractor.recreate_frames_from_trajectory(trajectory=trajectory)

print(f"Length of original frames: {len(frames)}")
print(type(frames))
print(type(frames[0]))
print(f"Length of reconstructed frames: {len(frames2)}")


Second Video
Length of original frames: 750
<class 'list'>
<class 'numpy.ndarray'>
Length of reconstructed frames: 750


In [ ]:
print(frames[0][0, :10])
print(frames2[0][0, :10])

np.array_equal(frames[0], frames2[0])

differences = np.where(frames[0] != frames2[0])[:10]
ctr = 0
for index in zip(*differences):
    ctr += 1
    print("Difference at position {}: {} vs {}".format(index, frames[0][index], frames2[0][index]))

ctr

In [ ]:
# The code below is just to test out how efficiently others implemented the rgb_array render mode.
#env = gym.make('CartPole-v1', render_mode="rgb_array")
#
#video_extractor = VideoExtractor(env)
#start_time = time.time()
#
#frames, _ = video_extractor.record_for_n_seconds(seconds=10)
#end_time = time.time()
#print(f"Execution time for frame gen: {end_time - start_time} seconds")
#env.close()
#
#start_time = time.time()
#video_extractor.generate_video(frames)
#end_time = time.time()
#print(f"Execution time for video gen: {end_time - start_time} seconds")